<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto Part 3</font></h1>

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

print('Libraries imported.')

Libraries imported.


## Install necessary APIs

In [2]:
#Install beautiful soup
#!conda install beautifulsoup4
#!conda install lxml
#!conda install html5lib
#!conda install requests
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
print('APIs installed.')

APIs installed.


## Extract the table from the website

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [3]:
from bs4 import BeautifulSoup
import requests
wikiweb_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(wikiweb_link).text

soup=BeautifulSoup(source,'lxml')

#scape the website, get the table and convert to pandas dataframe
citytable=soup.find('table',class_="wikitable sortable")
dfcity = pd.read_html(str(citytable), header=0)[0]

# convert all columns to string
dfcity['Postcode'] = dfcity['Postcode'].astype(str)
dfcity['Borough'] = dfcity['Borough'].astype(str)
dfcity['Neighbourhood'] = dfcity['Neighbourhood'].astype(str)
dfcity.head()
#dfcity.shape

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Pre-process data

<ul>

<li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</a> </li>
<li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</a> </li>
<li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</a> </li>
<li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</a> </li>
<li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</a> </li>
<li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</a> </li>
  
</ul>

In [4]:
#1. remove data with 'Not assigned' on Borough
dfcity=dfcity.drop(dfcity[dfcity.Borough=='Not assigned'].index)

#2. combine data with same postcode.
dfcity=dfcity.groupby(['Postcode',"Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)).reset_index()

#3. If a cell has a borough but a Not assigned neighborhood, 
#then the neighborhood will be the same as the borough. 
#So for the 9th cell in the table on the Wikipedia page, 
#the value of the Borough and the Neighborhood columns will be Queen's Park.
dfcity['Neighbourhood'].loc[dfcity['Neighbourhood'].str.contains('Not assigned')]=dfcity['Borough']

#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
dfcity.shape


(103, 3)

## Get Longtitude and Latitude of each Neighborhood

### The geopy is not generating the right/complete coordinates, Export coordinates from the csv instead

In [5]:
#!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print ("csv fetched")

csv fetched


In [6]:
#Read csv
coorsourcedp = pd.read_csv("Geospatial_Coordinates.csv")

# Regenerate neighborhoods
column_names=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

# instantiate new dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Make sure the data types are correct
neighborhoods['Latitude'] = neighborhoods['Latitude'].astype(np.float64)
neighborhoods['Longitude'] = neighborhoods['Longitude'].astype(np.float64)

tlat=0.0
tlong=0.0

for data in dfcity.itertuples():
    dummydf=coorsourcedp[coorsourcedp['Postal Code']==data[1]]   
    tlat=dummydf['Latitude'].values[0]
    tlong=dummydf['Longitude'].values[0]
    #print(tlat)
    neighborhoods = neighborhoods.append({'Postcode': data[1],
                                        'Borough': data[2],
                                        'Neighborhood': data[3],
                                        'Latitude': tlat,
                                        'Longitude': tlong}, ignore_index=True)


In [7]:
neighborhoods.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Explore and Cluster the neighborhoods in Toronto.

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

<li> to add enough Markdown cells to explain what you decided to do and to report any observations you make.</li>
<li> to generate maps to visualize your neighborhoods and how they cluster together.</li>

In [8]:
#Import folium

import folium # map rendering library

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Explore only bouroughs that contains the word 'Toronto'

In [10]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [11]:
torontoneighs= neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
torontoneighs.head()
torontoneighs.shape

(38, 5)

### Create the map of toronto

In [12]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, torontoneigh in zip(torontoneighs['Latitude'], torontoneighs['Longitude'], torontoneighs['Neighborhood'],torontoneighs['Borough']):
    label = '{}, {}'.format(torontoneigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Explore Neighborhoods in Toronto

Initialize variables and import needed libraries

In [13]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

CLIENT_ID = '43T434JZQOA3SSDEKRFGYOSXOFH3DAJBFHU0MWEBLHTM0DVB' # your Foursquare ID
CLIENT_SECRET = 'SOFPDSOSEPSHU5AV2VVPFDV5QEQMBRQM5KQT2FORBP2XH2IV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 43T434JZQOA3SSDEKRFGYOSXOFH3DAJBFHU0MWEBLHTM0DVB
CLIENT_SECRET:SOFPDSOSEPSHU5AV2VVPFDV5QEQMBRQM5KQT2FORBP2XH2IV


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get all nearbys of each neighborhood

In [16]:
toronto_venues = getNearbyVenues(names=torontoneighs['Neighborhood'],
                                   latitudes=torontoneighs['Latitude'],
                                   longitudes=torontoneighs['Longitude']
                                  )
print(toronto_venues.shape)

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

Check count for each neighborhood

In [17]:
#toronto_venues.groupby('Neighborhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


## Analyze Each Neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.rename(columns={'Neighborhood': 'Neigh'},inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
#toronto_onehot.head()



Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped.head()

### Let's print each neighborhood along with the top 5 most common venues

In [21]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.04
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2             Pub  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton,Exhibition Place,Parkdale Village----
                venue  freq
0      Breakfast Spot  0.09
1         Coffee Shop  0.09
2                Café  0.09
3       Burrito Place  0.05
4  Falafel Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Loun

###  Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Thai Restaurant,Café,Steakhouse,American Restaurant,Hotel,Restaurant,Burger Joint,Bar,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Steakhouse,Farmers Market,Pub,Café,Restaurant,Bakery
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden Center,Brewery,Farmers Market,Fast Food Restaurant,Spa,Burrito Place,Restaurant,Recording Studio
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Sculpture Garden,Plane,Harbor / Marina,Boutique,Airport Gate,Airport Food Court
5,"Cabbagetown,St. James Town",Coffee Shop,Park,Restaurant,Café,Bakery,Pizza Place,Italian Restaurant,Pub,Gift Shop,Butcher
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Bar,Sandwich Place,Thai Restaurant,Sushi Restaurant
7,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Dumpling Restaurant,Gaming Cafe
8,Christie,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Athletics & Sports,Diner,Nightclub,Coffee Shop,Convenience Store
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gym,Fast Food Restaurant,Pub,Men's Store


## Cluster Neighborhoods

In [24]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering
# run k-means clustering
kmeans = KMeans(init="random", n_clusters=kclusters, n_init=12)
kmeans=kmeans.fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 3, 1, 1, 3, 2,
       3, 4, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontoneighs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head()

In [26]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### First Cluster

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,0,Garden,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## 2nd Cluster

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Health Food Store,Coffee Shop,Pub,Neigh,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,East Toronto,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Bubble Tea Shop,Sports Bar,Japanese Restaurant,Spa,Juice Bar
2,East Toronto,1,Pub,Fast Food Restaurant,Steakhouse,Ice Cream Shop,Burrito Place,Burger Joint,Movie Theater,Fish & Chips Shop,Italian Restaurant,Sushi Restaurant
3,East Toronto,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Fish Market,Latin American Restaurant,Bookstore
5,Central Toronto,1,Gym,Food & Drink Shop,Park,Breakfast Spot,Clothing Store,Sandwich Place,Restaurant,Hotel,Burger Joint,Discount Store
6,Central Toronto,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park
7,Central Toronto,1,Sandwich Place,Dessert Shop,Café,Sushi Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Greek Restaurant,Deli / Bodega,Discount Store
9,Central Toronto,1,Coffee Shop,Pub,Fried Chicken Joint,American Restaurant,Sushi Restaurant,Bagel Shop,Sports Bar,Supermarket,Pizza Place,Convenience Store
11,Downtown Toronto,1,Coffee Shop,Park,Restaurant,Café,Bakery,Pizza Place,Italian Restaurant,Pub,Gift Shop,Butcher
12,Downtown Toronto,1,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gym,Fast Food Restaurant,Pub,Men's Store


## 3rd Cluster

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
25,Downtown Toronto,3,Café,Restaurant,Japanese Restaurant,Bakery,Bar,Bookstore,Coffee Shop,Beer Bar,Beer Store,Sandwich Place
26,Downtown Toronto,3,Café,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Dumpling Restaurant,Gaming Cafe
31,West Toronto,3,Supermarket,Bakery,Pharmacy,Gym / Fitness Center,Middle Eastern Restaurant,Discount Store,Bank,Bar,Furniture / Home Store,Pool
32,West Toronto,3,Bar,Coffee Shop,Asian Restaurant,Bakery,Restaurant,Pizza Place,Cocktail Bar,Café,New American Restaurant,Vietnamese Restaurant
34,West Toronto,3,Café,Bar,Mexican Restaurant,Grocery Store,Arts & Crafts Store,Sandwich Place,Bookstore,Fast Food Restaurant,Flea Market,Speakeasy


## 4th Cluster

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Restaurant,Gym,Playground,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


# Thank You!!